In [2]:
import torch
import numpy as np

In [2]:
from packages.Adam.Adam import FiniteDiffAdam, ParallelSpectrumOptimizer
from packages.rl.env_th import (
    least_sq_std_rew_and_c_th,
    load_coeffs_packed_split,
    generate_gaussian_complex_points
)

ImportError: cannot import name 'load_coeffs_packed_split' from 'packages.virasoro.virasoro_rec' (c:\Users\User\Desktop\git\ConformalBootstrapRL\packages\virasoro\virasoro_rec.py)

In [ ]:
z, zbar = generate_gaussian_complex_points(64, device="cuda")

# 3. 设置边界和参数
d = 9
B = 32
x_lo = torch.zeros(d, dtype=torch.float64)
x_hi = torch.ones(d, dtype=torch.float64)
fixed_vars = [1]

# 4. 实例化并运行 ParallelSpectrumOptimizer
optimizer = ParallelSpectrumOptimizer(
    d=d,
    h_step_obs=0.05,
    fixed_vars=fixed_vars,
    z=z,
    zbar=zbar,
    N_lsq=20,
    kmax=10,
    n_states_rew=3,
    device="cuda",
    beta1=0.9,
    beta2=0.999,
    eps=1e-8,
    buffer_threshold=0.01,
    use_sobol=True,
    sobol_seed=42
)

optimizer.run(
    B=B,
    x_lo=x_lo,
    x_hi=x_hi,
    lr=1e-2,
    steps_per_wave=50,
    num_waves=10
)

specs, rews = optimizer.get_buffer()
print("Buffer size:", len(specs))
for i, (s, r) in enumerate(zip(specs, rews)):
    print(f"{i}: reward = {r:.6f}, spectrum = {s.cpu().numpy()}")


NameError: name 'generate_gaussian_complex_points' is not defined

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from packages.rl.env_th import (
    least_sq_std_rew_and_c_th,
    load_coeffs_packed_split,
    generate_gaussian_complex_points,
    generate_gaussian_points_positive
)

# ─── Batched FD‐GD with correct grouping, custom initial points ──────────────
def fd_gradient_ascent_batched(
    bounds,               # [(x_min,x_max),(y_min,y_max)]
    phys_state,           # full 3-D state tensor
    coeffs,               # your virasoro coeffs
    B=50,                 # # trajectories
    lr=1e-2,              # step size
    h=5e-2,               # FD step
    num_steps=50,         # iterations
    device="cuda",
    initial_points=None,  # shape (B,2) or None
):
    dtype = phys_state.dtype
    lo2 = torch.tensor([bounds[0][0], bounds[1][0]], device=device, dtype=dtype)
    hi2 = torch.tensor([bounds[0][1], bounds[1][1]], device=device, dtype=dtype)

    # 1) init B starts in the 2-D box
    if initial_points is not None:
        X = torch.as_tensor(initial_points, device=device, dtype=dtype).clone()
        assert X.shape == (B, 2)
    else:
        X = lo2 + (hi2 - lo2) * torch.rand(B, 2, device=device, dtype=dtype)
    traj = torch.empty((num_steps+1, B, 2), device=device, dtype=dtype)
    traj[0] = X

    rews = torch.empty((num_steps+1, B), device=device, dtype=dtype)
    rews[0] = torch.zeros(B, device=device, dtype=dtype)

    eye2 = torch.eye(2, device=device, dtype=dtype)  # [[1,0],[0,1]]

    for t in range(num_steps):
        Xp = X.unsqueeze(1) + h*eye2.unsqueeze(0)  # (B,2,2)
        Xm = X.unsqueeze(1) - h*eye2.unsqueeze(0)  # (B,2,2)
        X_all = torch.cat([
            X,
            Xp[:,0,:], Xm[:,0,:],
            Xp[:,1,:], Xm[:,1,:]
        ], dim=0)

        N = X_all.size(0)
        S = torch.empty((N,3), device=device, dtype=dtype)
        S[:,0] = X_all[:,0]            # d1
        S[:,1] = phys_state[1]         # fixed d2
        S[:,2] = X_all[:,1]            # d3

        z, zbar = generate_gaussian_complex_points(400, device=device)
        Rf, _, _ = least_sq_std_rew_and_c_th(
            S, z, zbar, coeffs,
            N_lsq=20, kmax=10, kmax1=10,
            n_states_rew=3,
            device=device
        )
        R = Rf.view(5, B).transpose(0,1)  # (B,5)

        G = torch.zeros_like(X)
        G[:,0] = (R[:,1] - R[:,2]) / (2*h)
        G[:,1] = (R[:,3] - R[:,4]) / (2*h)

        X = X + lr * G

        mask_lo = X < lo2
        X = torch.where(mask_lo, 2*lo2 - X, X)
        mask_hi = X > hi2
        X = torch.where(mask_hi, 2*hi2 - X, X)
        X = torch.max(torch.min(X, hi2), lo2)

        traj[t+1] = X
        rews[t+1] = R[:,0]

    return traj.cpu().numpy().transpose(1,0,2), rews.cpu().numpy().transpose(1,0)  # (B, T+1, 2), (B,)

def run_and_plot_fdgd(
    phys_state,
    bounds2,
    coeffs,
    B=200,
    lr=1e-3,
    h=5e-2,
    steps=50,
    device="cuda",
    dtype=torch.float64,
    contour_std=0.1,
    kmax=10,
    kmax1=10,
    n_states_rew=3,
    initial_points=None
):
    

    # Run FD-GD
    trajs, rews = fd_gradient_ascent_batched(
        bounds2, phys_state, coeffs,
        B=B, lr=lr, h=h, num_steps=steps,
        device=device,
        initial_points=initial_points
    )

    # Make contour of the real reward on the (d1,d3) plane
    nx, ny = 200, 200
    xs = np.linspace(bounds2[0][0], bounds2[0][1], nx)
    ys = np.linspace(bounds2[1][0], bounds2[1][1], ny)
    Xg, Yg = np.meshgrid(xs, ys, indexing="xy")

    pts = torch.from_numpy(np.stack([Xg.ravel(), Yg.ravel()],1)).to(device, dtype=dtype)
    Sgrid = torch.empty((pts.size(0),3), device=device, dtype=dtype)
    Sgrid[:,0] = pts[:,0]
    Sgrid[:,1] = phys_state[1]
    Sgrid[:,2] = pts[:,1]
    z, zbar = generate_gaussian_points_positive(400, std=contour_std, device=device)
    Rf, _, _ = least_sq_std_rew_and_c_th(
        Sgrid, z, zbar, coeffs,
        N_lsq=20, kmax=kmax, kmax1=kmax1,
        n_states_rew=n_states_rew,
        device=device
    )
    Rg = Rf.cpu().numpy().reshape(nx,ny)

    fig, ax = plt.subplots(figsize=(6,6))
    cf = ax.contourf(Xg, Yg, Rg, levels=50, cmap="viridis")
    fig.colorbar(cf, label="reward")

    for b in range(B):
        ax.plot(trajs[b,:,0], trajs[b,:,1], '-k', alpha=0.3)
        ax.scatter(trajs[b,0,0],  trajs[b,0,1],  c='r', s=20)
        ax.scatter(trajs[b,-1,0], trajs[b,-1,1], c='b', s=20)

    ax.set_xlabel("d₁")
    ax.set_ylabel("d₃")
    ax.set_title(f"{B} batched FD‐GD runs w/ reflective BC (correct FD grouping)")
    plt.tight_layout()
    plt.show()

    return trajs, rews

def select_peaks(endpoints, rewards, rmin=0.1, n_peaks=6):
    """
    Select up to n_peaks points with the highest rewards, separated by at least rmin in L2 norm.

    Args:
        endpoints (np.ndarray): shape (B, D), where D is the dimension (2 or 3)
        rewards (np.ndarray): shape (B,)
        rmin (float): minimum distance between selected points
        n_peaks (int): number of peaks to select

    Returns:
        selected (np.ndarray): shape (n_peaks, D)
        selected_rewards (np.ndarray): shape (n_peaks,)
    """
    endpoints = np.asarray(endpoints)
    rewards = np.asarray(rewards)
    selected = []
    selected_rewards = []

    endpoints_copy = endpoints.copy()
    rewards_copy = rewards.copy()

    for _ in range(n_peaks):
        if len(rewards_copy) == 0:
            break
        idx = np.argmax(rewards_copy)
        best_point = endpoints_copy[idx]
        best_reward = rewards_copy[idx]
        selected.append(best_point)
        selected_rewards.append(best_reward)
        dists = np.linalg.norm(endpoints_copy - best_point, axis=1)
        mask = dists >= rmin
        endpoints_copy = endpoints_copy[mask]
        rewards_copy = rewards_copy[mask]

    selected = np.array(selected)
    selected_rewards = np.array(selected_rewards)

    print("Selected endpoints:\n", selected)
    print("Selected rewards:\n", selected_rewards)
    return selected, selected_rewards


device = "cuda"
dtype  = torch.float64
coeffs     = load_coeffs_packed_split("virasoro_coeffs.json")

In [ ]:
# 1) 定义搜索空间维度 d，以及 Sobol 或随机初始点 X0
B = 32     # 并行轨迹数
d = 3      # 示例维度，这里举例 3 维
# 假设状态空间每维在 [0, 1] 之间
X0 = torch.rand((B, d), dtype=torch.float64)  # 随机初始化，也可用 Sobol 或者特定初始点
x_lo = torch.zeros(d, dtype=torch.float64)
x_hi = torch.ones(d, dtype=torch.float64)

# 2) 固定某些坐标，例如 index=1 的坐标不变
fixed_vars = [1]

# 3) h_step_obs, z, zbar, coeffs, N_lsq, kmax, n_states_rew
h_step_obs = 0.05
# 这里举例创建 z, zbar；你实际使用中用 generate_gaussian_complex_points 生成
z = torch.linspace(0.1, 0.9, steps=64, dtype=torch.float64).to("cuda")
zbar = torch.conj(z)  # 仅示例，实际 zbar 由 generate_gaussian_complex_points 返回
# coeffs 由你的预先计算或加载，这里示例一个空对象
coeffs = None  # 你需要替换成实际 OPE coefficients 数据结构
N_lsq = 20
kmax = 10
n_states_rew = 3

# 4) 实例化优化器
optimizer = FiniteDiffAdam(
    h_step_obs=h_step_obs,
    fixed_vars=fixed_vars,
    z=z,
    zbar=zbar,
    coeffs=coeffs,
    N_lsq=N_lsq,
    kmax=kmax,
    n_states_rew=n_states_rew,
    device="cuda",
    beta1=0.9,
    beta2=0.999,
    eps=1e-8
)

# 5) 设定 Adam 的学习率和迭代步数
lr = 1e-2
num_steps = 50

# 6) 运行优化
best_X, best_R = optimizer.optimize(
    X0=X0,
    x_lo=x_lo,
    x_hi=x_hi,
    lr=lr,
    num_steps=num_steps
)

print("Best X per trajectory:", best_X)
print("Best reward per trajectory:", best_R)